### 종합실습1

Requests 혹은 Selenium 모듈을 이용해 부동산114 홈페이지로 접속한 다음, <br>
원하는 지역의 아파트 매물 시세 100건을 검색하여 주피터 노트북 화면에 출력하거나 엑셀로 저장해 보세요.<br>
(링크 : https://www.r114.com/)

#### requests모듈

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 홈페이지 오른쪽 상단 돋보기클릭 -포항 검색-아파트 클릭-매매 클릭
url = "https://www.r114.com/search/detailSearch.asp"
search_key = "포항"

results_list = []
for i in range(1, 11):
    print("{}페이지 크롤링 중입니다....".format(i))
    d = {
        "dqSearchTerm": search_key,
        "dqSearchTermOld": search_key,
        "dqSearchType": "memul",
        "dqOption": "apt",
        "dqSearchSubType": "매매",
        "dqPage": i
        }
    res = requests.post(url, data=d, verify=False)
    #response 헤더의 Content-Type에 charset 정보가 없음. 이로 인해 response된 데이터가 ISO-8859-1으로
    #인코딩된 것으로 requests 모듈이 자동판단, res.text 값을 ISO-8859-1로 디코딩해버림
    #하지만 실제로는 UTF-8로 인코딩되어 있으므로 UTF-8로 디코딩해야 한글이 정상 출력됨
    #print(res.encoding) # -> 'ISO-8859-1' 출력됨
    res.encoding="UTF-8"
    res_en = res.text
    soup = BeautifulSoup(res_en, "html.parser")
    datas = soup.select(".list_article > li")
    for data in datas:
        name = data.select(".tit_a > span:nth-child(1)")[0].text
        price = data.select(".tit_a > em")[0].text.split("매매")[-1].replace(" ", "")
        loc = data.select("span.txt > strong")[0].text
        loc_list = data.select("span.txt")[0].text.split(loc)[-1]
        feature = loc_list[:35].replace(" ", "")
        enviro = data.select("span.txt > em")[0].text
        addr = data.select(".txt p")[0].text[2:]
        # size = data.select(".txt")[0].text.split("|")[1]
        com = data.select("em.tel p.name")[0].text.split("C")[-1]
        phone = data.select("em.tel span")[0].text.split("전화번호")[-1]
        result = {
            "아파트명" : name,
            "매매가" : price,
            "특징" : feature,
            "주변환경" : enviro,
            "주소" : addr,
            "부동산" : com,
            "연락처" : phone
            }
        results_list.append(result)
    df = pd.DataFrame(results_list)
    df_ex = df.to_excel("./outputs/{}지역 아파트 매매가_requests모듈.xlsx".format(search_key), index=False)

#### selenium모듈

In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

#아래에 코드 작성
driver = webdriver.Chrome("chromedriver.exe")
driver.get("https://www.r114.com/")
driver.find_element(By.CSS_SELECTOR, "#dqSearchTerm").send_keys("포항")
driver.find_element(By.CSS_SELECTOR, "#cmdSearchHome").click()
driver.find_element(By.CSS_SELECTOR, ".list_tab_sale > .apt > a").click()
time.sleep(5)
results = []
for i in range(1, 11):
    html_text = driver.page_source
    soup = BeautifulSoup(html_text, "html.parser")
    items = soup.select(".list_article > li")
    for item in items:
        size_list = item.select(".txt")[0].text.split(" | ")
        size = [x for x in size_list if "㎡" in x][0]
        this_result_dict = {
            "아파트명" : item.select(".tit_a > span")[0].text,
            "지역" : item.select(".txt > strong")[0].text,
            "크기" : size,
            "매매가" : item.select(".tit_a > em")[0].text.replace("매매", "")
        }
        results.append(this_result_dict)
    if i < 10:
        for page_no in driver.find_elements(By.CSS_SELECTOR, ".paging > a"):
            if page_no.text.strip() == str(i+1):
                page_no.click()
                break
    
df = pd.DataFrame(results)
df.to_excel("../outputs/포항시 아파트 매매가 동향.xlsx", index = False)
df